# Task 1 - Exploration of customer purchasing behavior

In [6]:
# importing libraries
import warnings
warnings.filterwarnings("ignore")
#Data Manipulation and Treatment
import numpy as np
import pandas as pd
from datetime import datetime
#Plotting and Visualizations
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from scipy import stats
import itertools
#dvc
import dvc.api
import mlflow


In [8]:
#utils 
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from plotsClass import Plot

FileNotFoundError: [Errno 2] No such file or directory: '/home/neba/Desktop/Forecasting-Sales/Log/logs.log'